In [ ]:
import os
os.chdir("/home/tim/Development/OCPPM/")
from typing import Any, Callable
import pickle
import re
import string
from sklearn.preprocessing import StandardScaler

# Object centric process mining
import ocpa.objects.log.importer.ocel.factory as ocel_import_factory
from ocpa.algo.predictive_monitoring.obj import Feature_Storage
from ocpa.objects.log.ocel import OCEL
import ocpa.algo.predictive_monitoring.factory as feature_factory
from ocpa.algo.predictive_monitoring import tabular
import ocpa.objects.log.importer.csv.factory as csv_import_factory

In [ ]:
objects_metadata = {
    "customers": ["age", "bankaccount"],
    "items": [],
    "orders": [],
    "packages": [],
    "products": ["cost", "producer"],
}
event_attributes = ["weight", "price"]
ocel_in_file = "data/OTC/source/running-example.jsonocel"
feature_storage_intermediate_file = "data/OTC/feature_encodings/EFG/efg/raw/OTC_[P2_P3_O3_eas].fs"
feature_storage_out_file = "data/OTC/feature_encodings/EFG/efg/raw/OTC_split_[P2_P3_O3_eas].fs"

In [ ]:
ocel = ocel_import_factory.apply(ocel_in_file)

In [ ]:
all_acts = ocel.log.log["event_activity"].unique()
event_level_features = (
    {
        f"C2{char}": (feature_factory.EVENT_PRECEDING_ACTIVITIES, (act,))
        for act, char in zip(all_acts, string.ascii_lowercase)
    }
    | {
        "P2": (feature_factory.EVENT_ELAPSED_TIME, ()),
        "P3": (
            feature_factory.EVENT_REMAINING_TIME,
            (),
        ),  # only use for target (but we use 'ea4' now)
        # "P4": (feature_factory.EVENT_FLOW_TIME, ()),
        "P5": (feature_factory.EVENT_SYNCHRONIZATION_TIME, ()),
        # "P6": (feature_factory.EVENT_SOJOURN_TIME, ()),
    }
    | {
        f"O3{char}": (feature_factory.EVENT_PREVIOUS_TYPE_COUNT, (ot,))
        for ot, char in zip(objects_metadata, string.ascii_lowercase)
    }
)

In [ ]:
feature_storage = feature_factory.apply(
    ocel,
    event_based_features=list(event_level_features.values()),
    event_attributes=[f"event_{ea}" for ea in event_attributes],
    workers=1
)  # takes much too long when 'customers' is an object type

In [ ]:
with open(feature_storage_intermediate_file, "wb") as bin:
    pickle.dump(feature_storage, bin)

In [ ]:
feature_storage.extract_normalized_train_test_split(
    scaler=StandardScaler,
    scaling_exempt_features=[],
    test_size=0.3,
    validation_size=0.2,
    state=42,
)

In [ ]:
with open(feature_storage_out_file, "wb") as bin:
    pickle.dump(feature_storage, bin)